### Group Project Phase #2

For our predictive question, we are asking if specific quantitative characteristics of a track (danceability, energy, tempo) can help predict a new song's popularity on the music streaming service, Spotify.

Idea for Phase #2: Conduct classification analyses with ll combinations of predictive variables to see which results in the smallest RSME

In [22]:
library(tidyverse)
library(repr)
library(tidymodels)
library(cowplot)
library(RColorBrewer)

options(repr.plot.width= 30, repr.plot.height= 10)

In [23]:
set.seed(2022)

spotify_raw_data = read_csv("https://raw.githubusercontent.com/chrislukita/DSCI100-Group-Final/main/data/SpotifyDataset.csv")

Parsed with column specification:
cols(
  genre = col_character(),
  artist_name = col_character(),
  track_name = col_character(),
  track_id = col_character(),
  popularity = col_double(),
  acousticness = col_double(),
  danceability = col_double(),
  duration_ms = col_double(),
  energy = col_double(),
  instrumentalness = col_double(),
  key = col_character(),
  liveness = col_double(),
  loudness = col_double(),
  mode = col_character(),
  speechiness = col_double(),
  tempo = col_double(),
  time_signature = col_character(),
  valence = col_double()
)

Warning message:
“1 parsing failure.
   row col   expected     actual                                                                                             file
115177  -- 18 columns 15 columns 'https://raw.githubusercontent.com/chrislukita/DSCI100-Group-Final/main/data/SpotifyDataset.csv'
”


In [24]:
random_spotify = slice_sample(spotify_raw_data, n = 50000)

random_spotify

genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>
Pop,Lady Gaga,Always Remember Us This Way,2rbDhOo9Fh61Bbu23T2qCk,88,2.99e-01,0.553,210200,0.502,0.00e+00,C,0.7640,-5.972,Major,0.0409,129.976,4/4,0.2960
Children’s Music,PVRIS,Heaven,7eN3lGwu6SSmwYGH9PlpYa,52,1.89e-02,0.450,254358,0.840,2.51e-02,F,0.1390,-5.938,Major,0.0713,124.821,4/4,0.1850
Folk,Tom Walker,Fade Away,2ACoyK6t8MYwWI3G0WTV6r,56,8.25e-01,0.414,283989,0.267,0.00e+00,C,0.1600,-9.580,Minor,0.0306,133.274,4/4,0.1510
Hip-Hop,Phora,I Think I Love You,04WttLcLKlsuxYxf0hvouB,61,6.07e-01,0.544,210452,0.446,0.00e+00,F#,0.0923,-11.735,Major,0.5360,115.192,5/4,0.4690
Rap,Gucci Mane,Both Eyes Closed (feat. 2 Chainz and Young Dolph),5HtziFfFGxXpugCqZaAHoY,64,3.79e-01,0.704,260009,0.558,1.11e-04,C#,0.4070,-6.042,Major,0.2870,163.964,4/4,0.4140
Alternative,Serj Tankian,Sky Is Over,288ogAZr7aL3UIrd9CgZVy,51,3.78e-02,0.549,177000,0.888,7.22e-06,F,0.1560,-3.680,Minor,0.0291,104.976,4/4,0.3860
Children’s Music,In Flames,"I, The Mask",4sqfZMFTXNGWdHbkETHv9q,50,2.67e-05,0.470,221387,0.988,1.54e-02,A#,0.3100,-4.133,Minor,0.0836,104.995,4/4,0.4630
Country,Rascal Flatts,Backwards - Acoustic,1jAzxGqOqdgzwg1cxD6QgY,39,4.47e-01,0.722,203987,0.763,0.00e+00,C,0.2200,-7.684,Major,0.0910,122.931,4/4,0.3260
Children’s Music,Volbeat,Dead But Rising,3siZ8Gr2G9zelmm1NZOmP3,49,5.04e-06,0.395,214560,0.977,5.51e-01,G,0.3250,-5.572,Major,0.0487,107.692,4/4,0.5090


In [25]:
spotify_wrangled = random_spotify %>%
                    select(popularity: instrumentalness, liveness, loudness, speechiness, tempo, valence) %>%
                    na.omit(spotify_wrangled)

In [15]:
spotify_wrangled = spotify_wrangled %>%
                    mutate(popular = popularity >= 70) %>%
                    mutate(popular = as.factor(popular))

levels(spotify_wrangled$popular)[2] <- 'Yes'
levels(spotify_wrangled$popular)[1] <- 'No'

In [17]:
spotify_wrangled

popularity,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popular
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
88,2.99e-01,0.553,210200,0.502,0.00e+00,0.7640,-5.972,0.0409,129.976,0.2960,Yes
52,1.89e-02,0.450,254358,0.840,2.51e-02,0.1390,-5.938,0.0713,124.821,0.1850,No
56,8.25e-01,0.414,283989,0.267,0.00e+00,0.1600,-9.580,0.0306,133.274,0.1510,No
61,6.07e-01,0.544,210452,0.446,0.00e+00,0.0923,-11.735,0.5360,115.192,0.4690,No
64,3.79e-01,0.704,260009,0.558,1.11e-04,0.4070,-6.042,0.2870,163.964,0.4140,No
51,3.78e-02,0.549,177000,0.888,7.22e-06,0.1560,-3.680,0.0291,104.976,0.3860,No
50,2.67e-05,0.470,221387,0.988,1.54e-02,0.3100,-4.133,0.0836,104.995,0.4630,No
39,4.47e-01,0.722,203987,0.763,0.00e+00,0.2200,-7.684,0.0910,122.931,0.3260,No
49,5.04e-06,0.395,214560,0.977,5.51e-01,0.3250,-5.572,0.0487,107.692,0.5090,No


In [18]:
spotify_split = initial_split(spotify_wrangled, prop = 0.75, strata = popular)

spotify_train = training(spotify_split)
spotify_test = testing(spotify_split)

Tuning model to find best K value using two predictors

In [19]:
spotify_recipe <- recipe(popular ~ danceability + energy, data = spotify_train)

knn_tune <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) %>%
            set_engine("kknn") %>%
            set_mode("classification")

spotify_vfold <- vfold_cv(spotify_train, v = 5, strata = popular)

spotify_results <- workflow() %>%
                      add_recipe(spotify_recipe) %>%
                      add_model(knn_tune) %>%
                      tune_grid(resamples = spotify_vfold, grid = 10) %>%
                    collect_metrics()

accuracies <- spotify_results %>% 
        filter(.metric == "accuracy")

neighbors,.metric,.estimator,mean,n,std_err,.config
<int>,<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>
2,accuracy,binary,0.9196533,5,0.0009983097,Model1
2,roc_auc,binary,0.6674911,5,0.0033871684,Model1
5,accuracy,binary,0.9328267,5,0.0013369285,Model2
5,roc_auc,binary,0.6682801,5,0.0043492171,Model2
6,accuracy,binary,0.9329067,5,0.0013820436,Model3
6,roc_auc,binary,0.6692623,5,0.0035030905,Model3
7,accuracy,binary,0.9366133,5,0.0014026008,Model4
7,roc_auc,binary,0.6696240,5,0.0051766237,Model4
9,accuracy,binary,0.9387733,5,0.0012467914,Model5


Working with 9 neighbours

In [40]:
knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = 9) %>%
            set_engine("kknn") %>%
            set_mode("classification")

knn_fit = workflow() %>%
            add_recipe(spotify_recipe_3) %>%
            add_model(knn_spec) %>%
            fit(data = spotify_train)

testing_predicted_9 = predict(knn_fit, spotify_test) %>%
                bind_cols(spotify_test) %>%
                metrics(truth = popular, estimate = .pred_class) %>%
                filter(.metric == "accuracy") 

testing_predicted_9

.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
accuracy,binary,0.94048


Working with 15 neighbours

In [ ]:
knn_spec_2 <- nearest_neighbor(weight_func = "rectangular", neighbors = 15) %>%
            set_engine("kknn") %>%
            set_mode("classification")

knn_fit_2 = workflow() %>%
            add_recipe(spotify_recipe_3) %>%
            add_model(knn_spec_2) %>%
            fit(data = spotify_train)

testing_predicted_15 = predict(knn_fit, spotify_test) %>%
                bind_cols(spotify_test) %>%
                metrics(truth = popular, estimate = .pred_class) %>%
                filter(.metric == "accuracy") 

testing_predicted_15